In [1]:
LANGHCHAIN_TRACINB_V2 = "true"
LANGCHAIN_API_KEY = "ls__ef7c69da49cf48719e4d7c4b33a3debd"
import os
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY

In [2]:
import json
with open("train.json", "r") as file:
    dataset = json.load(file)
file.close()

In [3]:
for data in dataset[0]['annotations'][0]['result']:
    print(data['value']['text'])
    print('-'*100)

      IN THE HIGH COURT OF KARNATAKA,
          CIRCUIT BENCH AT GULBARGA

DATED THIS THE 22ND DAY OF FEBRUARY, 2013
----------------------------------------------------------------------------------------------------


      BEFORE

THE HON'BLE MR.JUSTICE ANAND BYRAREDDY

      CRIMINAL APPEAL NO.3532 OF 2012

BETWEEN:                                            R

Babu S/o Siddappa,                         .. APPELLANT
Age: 30 Years, Occ: Household,
R/o: Sunthan Village,
Taluk Chincholi,
District Gulbarga.

(By Shri Ishwar Raj S.Chowdapur, Advocate)

AND:

The State of Karnataka                     .. RESPONDENT
(Through Ratkal Police Station)
Represented by Additional State
Public Prosecutor, Circuit Bench,
Gulbarga.

(By Shri S.S.Aspalli, Government Pleader)

       
----------------------------------------------------------------------------------------------------
This Criminal Appeal is filed under Section 374(2) of the
Code of Criminal Procedure, 1973 by the advocate for the
app

In [4]:
from langchain_community.llms import Ollama
llm = Ollama(model='openchat')

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

In [6]:
len(dataset)

247

In [7]:
documents = []
document_labels = []
for data_part in dataset[1:100]:
    for data in data_part['annotations'][0]['result']:
        documents.append(Document(data['value']['text']))
        document_labels.append(data['value']['labels'])

In [8]:
len(document_labels)

11529

In [9]:
len(documents), documents

(11529,
 [Document(page_content="1/11\n\nIN THE HIGH COURT OF KARNATAKA, BENGALURU\n\n        DATED THIS THE 06TH DAY OF JULY 2018\n\n                                  PRESENT\n\n      THE HON'BLE Dr.JUSTICE VINEET KOTHARI\n\n                                      AND\n\n          THE HON'BLE Mrs.JUSTICE S.SUJATHA\n\n                           I.T.A.No.415/2017\n\nBETWEEN:\n\n1. PR. COMMISSIONER OF INCOME TAX-7\n         C.R. BUILDING, QUEENS ROAD\n         BANGALORE-560001.\n\n2. DEPUTY COMMISSIONER OF INCOME TAX\n         CIRCLE-5(1)(1), BANGALORE.\n                                                                     ...APPELLANTS\n\n(By Mr. E.I. SANMATHI, ADV.)\n\nAND:\n\nM/S. TESCO HINDUSTAN CENTRE PVT. LTD\nNo.81 & 82, EPIP AREA RESEARCH CENTRE-2\nWHITEFIELD, BANGALORE-560095\nPAN: AABCT8915B.\n\n                                                                 ...RESPONDENT\n(RESPONDETN SERVED & UNREPRESENTED)\n\n         "),
  Document(page_content="THIS I.T.A IS FILED UNDER SECTI

In [10]:
doc_label = {}
for i, doc in enumerate(documents):
    doc_label[doc.page_content] = document_labels[i]

In [11]:
embeddings = OllamaEmbeddings()

In [12]:
test = embeddings.embed_query('She had two daughters.')

In [13]:
query = "i have two daughters."
query_vector = embeddings.embed_query(query)

In [14]:
len(test), len(query_vector)

(4096, 4096)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([test], [query_vector])

array([[0.999132]])

In [16]:
# vector = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory='./embedding')

In [17]:
vector = Chroma(persist_directory='./embedding', embedding_function=embeddings)

In [18]:
sample_query = dataset[0]['annotations'][0]['result'][4]['value']['text']
sample_label = dataset[0]['annotations'][0]['result'][4]['value']['labels']
sample_query, sample_label

('\n       Heard the learned Counsel for the appellant and the learned Government Pleader.',
 ['NONE'])

In [19]:
vector.similarity_search(query=sample_query, k=3)

[Document(page_content='\n       Heard the learned counsel for the appellants and the learned State Public Prosecutor.'),
 Document(page_content='10. Heard the learned counsel for petitioner and learned Government Pleader for respondent State.'),
 Document(page_content='2. We have heard Mr. E.I. Sanmathi, learned Counsel for the Appellants  Revenue and Ms. Jinita Chatterjee, learned Counsel appearing for the Respondent.')]

In [20]:
doc_label['\n       Heard the learned counsel for the appellants and the learned State Public Prosecutor.'][0]

'NONE'

In [21]:
from tqdm import tqdm

In [30]:
true_labels = []
pred_labels = []
for data in tqdm(dataset[0]['annotations'][0]['result']):
    sample_query = data['value']['text']
    
    true_labels.append(data['value']['labels'][0])
    query_labels = []
    output = vector.similarity_search(query=sample_query, k=5)

    for out in output:
        query_labels.append([out.page_content, doc_label[out.page_content][0]])
    pred_labels.append(query_labels)

100%|██████████| 91/91 [00:07<00:00, 11.65it/s]


In [31]:
len(true_labels), len(pred_labels)

(91, 91)

In [32]:
pred_labels[0]

[['     IN THE HIGH COURT OF KARNATAKA\n                  DHARWAD BENCH\n\nDATED THIS THE 21ST DAY OF SEPTEMBER, 2015',
  'PREAMBLE'],
 ['     IN THE HIGH COURT OF KARNATAKA AT BENGALURU\n\n     DATED THIS THE 19TH DAY OF JANUARY 2016',
  'PREAMBLE'],
 ['\n\nIN THE HIGH COURT OF KARNATAKA, BENGALURU\n\n      DATED THIS THE 20TH DAY OF JULY 2018',
  'PREAMBLE'],
 ['                     \n\n         CIRCUIT BENCH AT DHARWAD\n\n   DATED THIS THE 29TH DAY OF JANUARY 2013',
  'PREAMBLE'],
 ['IN THE HIGH COURT OF KARNATAKA AT BENGALURU\n\nDATED THIS THE 26TH DAY OF FEBRUARY 2016',
  'PREAMBLE']]

In [33]:
pred_labels[30:35], '-'*30, true_labels[30:35]

([[['This witness also during his cross examination neither disclosed the dates and events of the harassment and cruelty nor such details were provided in the statement under Section 161 Code of Criminal Procedure, which would show that all the close family members being interested witnesses had made excoriated and material improvements in their testimony.',
    'ARG_PETITIONER'],
   ['Moreover, it was also contended that, it is a simple case of suicide but PW1 with the help of the testimonies of PW2 and PW4 has falsely implicated the appellant as an accused and these testimonies cannot be relied on as they were created as an afterthought after a delay of 2 days.',
    'ARG_PETITIONER'],
   ['When it was the specific case of the appellant that he had not committed sexual intercourse with the prosecutrix PW1 and when a defence was raised that she was pregnant from Vishnu s/o Ambadas , her maternal cousin, it was necessary for the investigating officer to have conducted DNA test to ascer

In [35]:
import joblib
joblib.dump(pred_labels, './vecs/top_preds')
joblib.dump(true_labels, './vecs/true_labels')
joblib.dump(doc_label, "./vecs/doc_label")

['./vecs/doc_label']

In [18]:
from sklearn.metrics import accuracy_score, f1_score
acc = accuracy_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels, average="macro")

In [19]:
acc, f1

(0.3076923076923077, 0.2922894072648573)

In [29]:
output, sample_query

('\n Sd/- JUDGE kmv', '\n Sd/- JUDGE nv')

In [30]:
test = embeddings.embed_query(output)
query = embeddings.embed_query(sample_query)
len(query), len(test)

(4096, 4096)

In [31]:
cosine_similarity([query], [test])

array([[0.82109967]])

In [ ]:
# vector store-->top k similar-->
# prompt the llm with labels of top k with sentences and the current sentence for inference along with the original prompt + definiation of RR
# provide additional information with the sentence like section or previous sentence